<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/AlbertBase_TransformationList_Custom_Recipe_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip installs

In [ ]:
pip install transformers

In [ ]:
!pip3 install textattack[tensorflow]

In [ ]:
pip install --upgrade numpy

In [ ]:
pip install folium==0.2.1

## Imports

In [3]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import ImprovedGeneticAlgorithm
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import (
   
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod

In [8]:
from textattack.datasets import HuggingFaceDataset
from textattack.attacker import Attacker
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import TargetedClassification
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
import torch

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")
# We wrap the model so it can be used by textattack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
#dataset = HuggingFaceDataset("imdb", split="test")
#dataset = HuggingFaceDataset("glue", "sst2", "train")

Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a. Subsequent calls will reuse this data.


##Custom Recipes

##Recipe1    
            WordSwapQWERTY
            WordSwapWordNet
            WordSwapHomoglyphSwap
            WordSwapNeighboringCharacterSwap
            WordSwapRandomCharacterInsertion
            WordSwapRandomCharacterDeletion
            WordSwapEmbedding
      

In [5]:
class Recipe(Attack,ABC):
    @staticmethod
    def build(model_wrapper,transf):
        if(transf == "WordSwapWordNet"):
            transformation = WordSwapWordNet()
        elif(transf == "WordSwapQWERTY"):
            transformation = WordSwapQWERTY()
        elif(transf == "WordSwapHomoglyphSwap"):
            transformation = WordSwapHomoglyphSwap()
        elif(transf == "WordSwapNeighboringCharacterSwap"):
            transformation = WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        elif(transf == "WordSwapRandomCharacterInsertion"):
            transformation = WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    )
        elif(transf == "WordSwapRandomCharacterDeletion"):
             transformation = WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    )
        elif(transf == "WordSwapEmbedding"):
             transformation = WordSwapEmbedding(max_candidates=5)

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = ImprovedGeneticAlgorithm(pop_size=60,
                   max_iters=20, temp=0.3, give_up_if_no_improvement=False, post_crossover_check=True, max_crossover_retries=20, max_replace_times_per_index=5)
        return Attack(goal_function, constraints, transformation, search_method)


In [17]:
transf_list = ['WordSwapQWERTY','WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
                      'WordSwapRandomCharacterDeletion','WordSwapEmbedding']
for transf in transf_list:
    print("Starting attack for-----------------",transf)
    attack = Recipe.build(model_wrapper,transf)
    attack_args = AttackArgs(num_examples=8)
    attacker = Attacker(attack, dataset,attack_args)
    attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/8 [00:00<?, ?it/s]

Starting attack for----------------- WordSwapQWERTY
Attack(
  (search_method): ImprovedGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  True
    (max_crossover_retries):  20
    (max_replace_times_per_index):  5
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapQWERTY
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 





 12%|█▎        | 1/8 [00:04<00:29,  4.16s/it]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  12%|█▎        | 1/8 [00:04<00:29,  4.16s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (83%)]]

lovingly photographed in the manner of a [[golden]] book sprung to life , [[stuart]] [[little]] 2 [[manages]] [[sweetness]] largely without stickiness .

lovingly photographed in the manner of a [[golfen]] book sprung to life , [[stuarg]] [[liftle]] 2 [[managds]] [[sweerness]] largely without stickiness .






[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  25%|██▌       | 2/8 [00:04<00:14,  2.43s/it]

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:  25%|██▌       | 2/8 [00:04<00:14,  2.43s/it]

[Succeeded / Failed / Skipped / Total] 1 / 1 / 1 / 3:  38%|███▊      | 3/8 [00:04<00:08,  1.63s/it]

...

--------------------------------------------- Result 5 ---------------------------------------------
[[Positive (74%)]] --> [[Negative (91%)]]

red dragon " never cuts [[corners]] .

red dragon " never cuts [[nooks]] .


--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (64%)]] --> [[[SKIPPED]]]

fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .






[Succeeded / Failed / Skipped / Total] 4 / 1 / 2 / 7:  88%|████████▊ | 7/8 [00:26<00:03,  3.72s/it]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (51%)]]

throws in enough [[clever]] and unexpected [[twists]] to [[make]] the [[formula]] [[feel]] [[fresh]] .

throws in enough [[shrewd]] and unexpected [[tendrils]] to [[rendering]] the [[wording]] [[believe]] [[costs]] .






[Succeeded / Failed / Skipped / Total] 4 / 1 / 2 / 7: 100%|██████████| 8/8 [00:26<00:00,  3.36s/it]

[Succeeded / Failed / Skipped / Total] 5 / 1 / 2 / 8: 100%|██████████| 8/8 [00:26<00:00,  3.36s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (97%)]] --> [[Negative (61%)]]

weighty and [[ponderous]] but every bit as filling as the treat of the title .

weighty and [[cumbersome]] but every bit as filling as the treat of the title .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 2      |
| Original accuracy:            | 75.0%  |
| Accuracy under attack:        | 12.5%  |
| Attack success rate:          | 83.33% |
| Average perturbed word %:     | 21.99% |
| Average num. words per input: | 15.0   |
| Avg num queries:              | 253.67 |
+-------------------------------+--------+
